# Collaboration Network Analysis

`Overview`
This notebook analyzes and visualizes collaboration networks between countries and institutions:
- Constructing network graphs from project partnership data
- Analyzing network metrics (centrality, clustering, community detection)
- Creating interactive network visualizations
- Identifying key collaboration patterns and influential nodes

`Inputs`
- Processed project and partnership data from `../data/processed/`

`Outputs`
- Network analysis results in `../data/results/`
- Interactive network visualizations in `../reports/figures/`

`Dependencies`
- NetworkX
- Matplotlib
- Plotly
- Pandas
- Community detection libraries

*Note: This is notebook 3 of the analysis pipeline focusing on collaboration patterns*